<a href="https://colab.research.google.com/github/SwaksharDeb/Activity-recognition/blob/master/HAR_paper1_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dependecies

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import array
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from numpy import savetxt
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
import scipy.stats as stats
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from math import sqrt
import math

# Import dataset

In [ ]:
"""Import Datasets"""
dataset_user04 = pd.read_csv("final_field_user04_dataset.csv")
dataset_user07 = pd.read_csv("final_field_user07_dataset.csv")
dataset_user08 = pd.read_csv("final_field_user08_dataset.csv")
dataset_user18 = pd.read_csv("final_field_user18_dataset.csv")
dataset_user38 = pd.read_csv("final_field_user38_dataset.csv")
dataset_user51 = pd.read_csv("final_field_user51_dataset.csv")
dataset_lab = pd.read_csv("lab data localized.csv")

dataset = pd.concat([dataset_lab, dataset_user04, dataset_user07, dataset_user08, dataset_user18, dataset_user38, dataset_user51], ignore_index = True)


# Data processing

In [ ]:
""" Data processing"""
X = dataset.iloc[:, [0, 1, 2]].values
Y = dataset.iloc[:, 3].values
Y = Y - 1

## Data Augmentation

we just use scaling for data augmentation. The minority classes are scaled and added to the original data to make the original data balance. Our assumption is that, if we scale a signal it still represent the same activity

In [ ]:
# Data Augmentation
scalling_factor = [0.3,0.5,0.7,0.8,1.2,1.5,2]

for scal in scalling_factor:
    X_1_scal = X[np.where(Y==1)]*scal
    label_1 = Y[np.where(Y==1)]
    X_2_scal = X[np.where(Y==2)]*scal
    label_2 = Y[np.where(Y==2)]
    X_5_scal = X[np.where(Y==5)]*scal
    label_5 = Y[np.where(Y==5)]
    X_7_scal = X[np.where(Y==7)]*scal
    label_7 = Y[np.where(Y==7)]
    X_8_scal = X[np.where(Y==8)]*scal
    label_8 = Y[np.where(Y==8)]
    X_9_scal = X[np.where(Y==9)]*scal
    label_9 = Y[np.where(Y==9)]
    X_10_scal = X[np.where(Y==10)]*scal
    label_10 = Y[np.where(Y==10)]
    X = np.concatenate((X, X_1_scal,X_2_scal,X_5_scal,X_7_scal,X_8_scal,X_9_scal,X_10_scal))
    Y = np.concatenate((Y, label_1,label_2, label_5, label_7, label_8, label_9, label_10))

scalling_factor = [0.5,2]

for scal in scalling_factor:
    X_3_scal = X[np.where(Y==3)]*scal
    label_3 = Y[np.where(Y==3)]
    X_11_scal = X[np.where(Y==11)]*scal
    label_11 = Y[np.where(Y==11)]
    X = np.concatenate((X, X_3_scal, X_11_scal))
    Y = np.concatenate((Y, label_3, label_11))

scalling_factor = [0.3,0.5,1.5,2]

for scal in scalling_factor:
    X_0_scal = X[np.where(Y==0)]*scal
    label_0 = Y[np.where(Y==0)]
    X_6_scal = X[np.where(Y==6)]*scal
    label_6 = Y[np.where(Y==6)]
    X = np.concatenate((X, X_0_scal, X_6_scal))
    Y = np.concatenate((Y, label_0, label_6))

In [ ]:
del X_1_scal

In [ ]:
X.shape,Y.shape

((26992376, 3), (26992376,))

## low pass filter

In [ ]:
low_pass_filter = np.zeros((X.shape[0], X.shape[1]))
for i in range(len(X)):
    for j in range(X.shape[1]):
        if i == 0:
            low_pass_filter[i][j] = X[i][j]
        else:
            low_pass_filter[i][j] = 0.8*X[i-1][j] + (1-0.8)*X[i][j]

## Frame preparation

In [ ]:
# frame preparation
Fs = 4
frame_size = Fs*20 # 80 # 20 seconds of data
hop_size =  Fs*10

def get_frames(df, frame_size, hop_size,label_data):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, len(df) - frame_size, hop_size):
        value = df[i: i + frame_size, :]
        #y = df[i: i + frame_size, 1]
        #z = df[i: i + frame_size, 2]

        # Retrieve the most often used label in this segment
        label = stats.mode(label_data[i: i + frame_size])[0][0]
        frames.append([value])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

row_frame, row_label = get_frames(X, frame_size, hop_size, Y)
low_pass_frame, low_pass_label = get_frames(low_pass_filter, frame_size, hop_size, Y)

In [ ]:
row_frame.shape, low_pass_frame.shape

((674808, 80, 3), (674808, 80, 3))

## Feature calculation

In [ ]:
#feature calculation
mean = np.mean(low_pass_frame, axis = 1)  # mean along (x,y,z) axis
std = np.std(low_pass_frame, axis = 1)  # standard deviation along (x,y,z) axis

rms_x = []  # root mean square along x axis
value = 0
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][0] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_x.append(value)
    value = 0

rms_y = []  # root mean square along y axis
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][1] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_y.append(value)
    value = 0

rms_z = []  # root mean square z axis
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][2] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_z.append(value)
    value = 0

In [ ]:
lengths = []  # length of the accelerometer vector
length = 0
nx, ny, nz = row_frame.shape[0], row_frame.shape[1], row_frame.shape[2]
acc_vector = np.reshape(row_frame,(nx, ny*nz))

for i in range(acc_vector.shape[0]):
    count = 0
    for j in range(acc_vector.shape[1]):
        length += acc_vector[i][j]**2
        count += 1

        if count == 3:
            length = sqrt(length)
            lengths.append(length)
            length = 0
            count = 0

avc = []  # avc feature

polling_rate = 4 #Hz

for i in range(0, len(lengths), frame_size):
    value = 0
    iterable = lengths[i:i+frame_size]
    for j in range(len(iterable) - 1, 1, -1):
        value += abs(iterable[j] - iterable[j - 1])

    value = value / (frame_size / polling_rate)
    avc.append(value)


In [ ]:
max_min = []
for i in range(0, len(lengths), frame_size):
    iterable = lengths[i:i+frame_size]

    max_min.append(max(iterable) - min(iterable))


In [ ]:
angle = np.zeros((low_pass_frame.shape[0],low_pass_frame.shape[1],low_pass_frame.shape[2]))  # inclenation angle
angles = []
length = 0
for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        length = low_pass_frame[i,j,0]**2 + low_pass_frame[i,j,1]**2 + low_pass_frame[i,j,2]**2
        length = sqrt(length)
        if length == 0:
            print("length is zero in index row= ",i,"column= ",j)
        else:
            angle_x = math.acos(low_pass_frame[i,j,0]/length)
            angle_y = math.acos(low_pass_frame[i,j,1]/length)
            angle_z = math.acos(low_pass_frame[i,j,2]/length)
            angle[i][j][0] = angle_x
            angle[i][j][1] = angle_y
            angle[i][j][2] = angle_z
nx, ny, nz = angle.shape
angle = np.reshape(angle,(nx,ny*nz))


In [ ]:
input_vector = np.zeros((row_frame.shape[0], angle.shape[1] + 11))  # input vector for the algorithm
for i in range(input_vector.shape[0]):
    input_vector[i][0] = mean[i][0]
    input_vector[i][1] = mean[i][1]
    input_vector[i][2] = mean[i][2]
    input_vector[i][3] = std[i][0]
    input_vector[i][4] = std[i][1]
    input_vector[i][5] = std[i][2]
    input_vector[i][6] = rms_x[i]
    input_vector[i][7] = rms_y[i]
    input_vector[i][8] = rms_z[i]
    input_vector[i][9] = max_min[i]
    input_vector[i][10] = avc[i]

for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        input_vector[i][j+11] = angle[i][j]

In [ ]:
input_vector.shape, row_label.shape

((674808, 251), (674808,))

In [ ]:
del angle 
del max_min
del avc
del mean
del rms_x
del rms_y
del rms_z
del lengths
del X_2_scal
del X_3_scal
del X_5_scal
del X_6_scal
del X_7_scal
del X_8_scal
del X_9_scal
del X_10_scal
del X_11_scal

# Standarization

In [19]:
sc = StandardScaler()
input_vector = sc.fit_transform(input_vector)

# Trian Test split

In [20]:
X_train, X_val, y_train, y_val = train_test_split(input_vector, row_label, test_size = 0.2, random_state = 0, stratify = row_label)

In [21]:
X_train.shape, y_train.shape

((539846, 251), (539846,))

# Algorithm

## Neural Network

In [33]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(X_train.shape[1], )))
#model.add(Dropout(0.2))
model.add(Dense(units=64, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=128, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(units=324, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=12, activation='softmax'))

In [34]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [35]:
history = model.fit(X_train, y_train, epochs = 50, batch_size =64, validation_data= (X_val, y_val), verbose=1)

Epoch 1/50
8436/8436 [==============================] - 20s 2ms/step - loss: 1.2150 - accuracy: 0.6254 - val_loss: 1.0627 - val_accuracy: 0.6630
Epoch 2/50
8436/8436 [==============================] - 22s 3ms/step - loss: 1.0003 - accuracy: 0.6781 - val_loss: 0.9527 - val_accuracy: 0.6911
Epoch 3/50
8436/8436 [==============================] - 22s 3ms/step - loss: 0.9062 - accuracy: 0.7076 - val_loss: 0.8804 - val_accuracy: 0.7173
Epoch 4/50
8436/8436 [==============================] - 21s 2ms/step - loss: 0.8469 - accuracy: 0.7275 - val_loss: 0.8225 - val_accuracy: 0.7382
Epoch 5/50
8436/8436 [==============================] - 20s 2ms/step - loss: 0.8050 - accuracy: 0.7410 - val_loss: 0.8167 - val_accuracy: 0.7403
Epoch 6/50
8436/8436 [==============================] - 22s 3ms/step - loss: 0.7759 - accuracy: 0.7506 - val_loss: 0.7850 - val_accuracy: 0.7484
Epoch 7/50
8436/8436 [==============================] - 21s 3ms/step - loss: 0.7535 - accuracy: 0.7583 - val_loss: 0.7509 - val_ac

In [36]:
y_pred = model.predict(X_val)

In [37]:
y_pred_ = np.zeros((y_pred.shape[0]))
for i in range(y_pred.shape[0]):
    y_pred_[i] = np.argmax(y_pred[i,:])

In [38]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred_, y_val)

In [39]:
pd.DataFrame(cm)

,0,1,2,3,4,5,6,7,8,9,10,11
0,741,67,112,173,83,78,138,11,15,16,11,22
1,91,5045,434,100,34,521,175,192,192,254,86,29
2,853,662,64925,2175,891,590,2843,278,258,367,171,365
3,99,34,213,3847,751,118,381,4,25,58,29,81
4,15,3,22,66,3210,23,42,1,0,5,0,15
5,118,138,392,892,118,12052,1177,97,169,88,62,45
6,126,37,503,732,226,376,4084,12,18,41,6,59
7,41,95,159,49,12,108,69,2961,196,93,153,20
8,45,156,351,102,29,248,125,219,4493,271,179,33
9,37,91,143,130,32,49,163,85,270,3967,134,33


In [40]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred_)

In [41]:
per_class_accuracy = {}
for i in range(cm.shape[0]):
    accuracy = cm[i,i]/sum(cm[i,:])
    per_class_accuracy[i] = accuracy

In [42]:
per_class_accuracy

{0: 0.5051124744376279,
 1: 0.7052984761638473,
 2: 0.8729059668181451,
 3: 0.6820921985815603,
 4: 0.9435626102292769,
 5: 0.7852488923638259,
 6: 0.6565916398713826,
 7: 0.7484833164812943,
 8: 0.7187649976003839,
 9: 0.7726918582002338,
 10: 0.7446431733663892,
 11: 0.8961748633879781}